In [1]:
#coding:utf-8
%matplotlib inline
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
import PIL.Image 
import scipy.misc
import tensorflow as tf

In [2]:
graph  = tf.Graph()
sess = tf.InteractiveSession(graph = graph)

In [3]:
model_fn = "inception5h/tensorflow_inception_graph.pb"

with tf.gfile.FastGFile(model_fn,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

Instructions for updating:
Use tf.gfile.GFile.


In [4]:
#定义输入图像的占位符
t_input = tf.placeholder(np.float32,name='input')
    
#图像预处理
#减均值
imagenet_mean = 117.0
    
#增加维度
t_preprocessed = tf.expand_dims(t_input - imagenet_mean,0)
    
#导入模型并将预处理的图像送入网络中
tf.import_graph_def(graph_def,{'input':t_preprocessed})

In [5]:
#找到卷积层
layers = [op.name for op in graph.get_operations() 
          if op.type=='Conv2D' and 'import/' in op.name]
print(len(layers))

59


In [ ]:
print(layers)

In [6]:
def savearray(img_array,img_name):
    scipy.misc.toimage(img_array).save(img_name)
    print('img saved %s' % img_name)

In [7]:
#将图像放大ratio
def resize_ratio(img,ratio):
    min = img.min()
    max = img.max()
    img = (img - min) / (max-min) *255
    img = np.float32(scipy.misc.imresize(img,ratio))
    img = img / 255 *(max - min) + min
    return img

In [15]:
#调整图像尺寸
def resize(img,hw):
    min = img.min()
    max = img.max()
    img = (img - min) / (max-min) *255
    img = np.float32(scipy.misc.imresize(img,hw))
    img = img / 255 *(max - min) + min
    return img

In [9]:
#原始图像尺寸可能很大,从而导致内存耗尽问题
#每次只对tile_ size * tile_ size大小的图像计算梯度，避免内存问
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h, w = img.shape[:2]
    sx,sy = np.random.randint(sz, size=2)
    img_shift = np.roll(np.roll(img, sx, 1),sy, 0) #先在行上做整体移动
    grad = np.zeros_like(img)
    
    for y in range(0, max(h- sz // 2, sz), sz):
        for x in range(0, max(w- sz// 2, sz), sz):
            sub = img_shift[y:y + sz, x:x + sz]
            g = sess.run(t_grad, {t_input: sub})
            grad[y:y+ sz,x:x + sz]= g
            
    return np.roll(np.roll(grad, -sx, 1), -sy, 0)

In [10]:
def render_native(t_obj,img0,iter_n=20,step=1.0):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score,t_input)[0]
    
    img = img0.copy()
    
    for i in range(iter_n):
        g,score = sess.run([t_grad,t_score],{t_input:img})
        g /= g.std() + 1e-8
        img += g*step
        print("iter:%d" % (i+1),' score(mean)=%f' % score)
              
     #保存图片
    savearray(img,'native_deepdream.jpg')

In [17]:
#图像的拉普拉斯金字塔分解
def render_deepdream(t_obj, img0,iter_n=10,step=1.5, 
                     octave_n=4, octave_scale=1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score, t_input)[0]  
    img = img0.copy()
    
    #将图像进行金字塔分解#从而分为高频、低频部分
    octaves = []
    for i in range(octave_n-1):
        hw = img.shape[:2]
        lo= resize(img, np.int32(np.float32(hw)/ octave_scale))
        hi= img - resize(lo, hw)
        img = lo
        octaves.append(hi)

    #首先生成低频的图像，再依次放大开加上高频
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]
            img = resize(img,hi.shape[:2])+hi
            
    for i in range(iter_n):
        g = calc_grad_tiled(img,t_grad)
        img += g* (step/ (np.abs(g).mean()+ 1e-7))

    img = img.clip(0, 255)
    savearray(img, 'mountain_deepdream.jpg')
    im = PIL.Image.open("mountain_deepdream.jpg").show()

In [12]:
#定义卷积层 通道数 并取出对应的tensor
name = 'mixed4d_3x3_bottleneck_pre_relu'
channel = 139

layer_output = graph.get_tensor_by_name('import/%s:0' % name)

#定义噪声图像
image_noise = np.random.uniform(size=(224,224,3))+100.0

#调用渲染函数
render_native(layer_output[:,:,:,channel],image_noise,iter_n=20)

#保存并显示图片
im = PIL.Image.open('native_deepdream.jpg')
im.show()
im.save('native_single_chn.jpg')

iter:1  score(mean)=-20.062153
iter:2  score(mean)=-30.255493
iter:3  score(mean)=14.069202
iter:4  score(mean)=83.265350
iter:5  score(mean)=153.393860
iter:6  score(mean)=213.695724
iter:7  score(mean)=273.637329
iter:8  score(mean)=311.356750
iter:9  score(mean)=361.026001
iter:10  score(mean)=414.331512
iter:11  score(mean)=460.313904
iter:12  score(mean)=490.162384
iter:13  score(mean)=529.660339
iter:14  score(mean)=563.659912
iter:15  score(mean)=591.177551
iter:16  score(mean)=627.152832
iter:17  score(mean)=646.298523
iter:18  score(mean)=674.900452
iter:19  score(mean)=685.804443
iter:20  score(mean)=718.683167
img saved native_deepdream.jpg


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


In [ ]:
#定义卷积层 通道数 并取出对应的tensor
name = 'mixed4d_3x3_bottleneck_pre_relu'

layer_output = graph.get_tensor_by_name('import/%s:0' % name)

image_noise = np.random.uniform(size=(224,224,3))+100.0

#调用渲染函数
#不指定通道
render_native(layer_output,image_noise,iter_n=20)

im = PIL.Image.open('native_deepdream.jpg')
im.show()
im.save('native_single_chn.jpg')

In [13]:
#定义卷积层 通道数 并取出对应的tensor
name = 'mixed4c'
layer_output = graph.get_tensor_by_name('import/%s:0' % name)

image_test = PIL.Image.open('test.jpeg')
#调用渲染函数
render_native(layer_output,image_test ,iter_n=30)

im = PIL.Image.open('native_deepdream.jpg')
im.show()
im.save('mountain_native.jpg')

iter:1  score(mean)=13.703018
iter:2  score(mean)=17.948713
iter:3  score(mean)=23.838718
iter:4  score(mean)=29.935328
iter:5  score(mean)=35.357555
iter:6  score(mean)=40.110874
iter:7  score(mean)=44.779991
iter:8  score(mean)=48.602577
iter:9  score(mean)=52.341274
iter:10  score(mean)=55.210251
iter:11  score(mean)=58.362148
iter:12  score(mean)=61.072186
iter:13  score(mean)=63.665562
iter:14  score(mean)=66.043480
iter:15  score(mean)=68.456337
iter:16  score(mean)=70.420029
iter:17  score(mean)=72.665192
iter:18  score(mean)=74.284988
iter:19  score(mean)=76.274757
iter:20  score(mean)=77.873222
iter:21  score(mean)=79.491112
iter:22  score(mean)=80.921425
iter:23  score(mean)=82.456726
iter:24  score(mean)=83.729187
iter:25  score(mean)=85.063194
iter:26  score(mean)=86.365059
iter:27  score(mean)=87.576828
iter:28  score(mean)=88.910416
iter:29  score(mean)=89.873627
iter:30  score(mean)=91.084976
img saved native_deepdream.jpg


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


In [18]:
#定义卷积层 通道数 并取出对应的tensor
name = 'mixed4c'
layout_output = graph.get_tensor_by_name('import/%s:0' % name)

image_test = PIL.Image.open('test.jpeg')
image_test = np.float32(image_test)

#调用渲染函数
render_deepdream(tf.square(layer_output),image_test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


img saved mountain_deepdream.jpg


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  
